In [1]:
import asyncio
import aiofile
import nest_asyncio

from amazon_transcribe.client import TranscribeStreamingClient
from amazon_transcribe.handlers import TranscriptResultStreamHandler
from amazon_transcribe.model import TranscriptEvent

In [2]:
"""
Here's an example of a custom event handler you can extend to
process the returned transcription results as needed. This
handler will simply print the text out to your interpreter.
"""
class MyEventHandler(TranscriptResultStreamHandler):
    async def handle_transcript_event(self, transcript_event: TranscriptEvent):
        # This handler can be implemented to handle transcriptions as needed.
        # Here's an example to get started.
        results = transcript_event.transcript.results
        for result in results:
            for alt in result.alternatives:
                print(alt.transcript)


async def basic_transcribe(file_path: str):
    # Set up our client with your chosen Region
    client = TranscribeStreamingClient(region = "eu-central-1")

    # Start transcription to generate async stream
    stream = await client.start_stream_transcription(
        language_code = "en-US",
        media_sample_rate_hz = 16000,
        media_encoding = "pcm",
    )

    async def write_chunks():
        # NOTE: For pre-recorded files longer than 5 minutes, the sent audio
        # chunks should be rate limited to match the real-time bitrate of the
        # audio stream to avoid signing issues.
        async with aiofile.AIOFile(file_path, 'rb') as afp:
            reader = aiofile.Reader(afp, chunk_size = 1024 * 16)
            async for chunk in reader:
                await stream.input_stream.send_audio_event(audio_chunk = chunk)
        await stream.input_stream.end_stream()

    # Instantiate our handler and start processing events
    handler = MyEventHandler(stream.output_stream)
    await asyncio.gather(write_chunks(), handler.handle_events())

In [3]:
nest_asyncio.apply()

loop = asyncio.get_event_loop()
loop.run_until_complete(basic_transcribe('../data/short_trump_biden_highlights.wav'))
loop.close()

Bang.
Bang.
Bang.
Bang.
Bangor.
Bangor.
Mm hmm.
Mm hmm.
Mm hmm.


KeyboardInterrupt: 

Yeah.
Yeah.
Yeah.
Hmm.
Hmm.
Hmm.
All right.
All right.
All right.
Hmm.
Hmm.
Hmm.
